# Cross Border Capacities

## Load Ember and Pypsa-Eur Data 

In [ ]:
import pandas as pd
import pypsa
import matplotlib.pyplot as plt

n = pypsa.Network(".../results/validation_2023/networks/base_s_39__79seg_2025.nc")
df = pd.read_csv(".../validation/ember_data/ntc.csv")


## Process Ember Data

In [ ]:
iso3_to_iso2 = {
    'ALB': 'AL', 'AUT': 'AT', 'BEL': 'BE', 'BGR': 'BG', 'BIH': 'BA',
    'CHE': 'CH', 'CYP': 'CY', 'CZE': 'CZ', 'DEU': 'DE', 'DNK': 'DK',
    'DZA': 'DZ', 'EGY': 'EG', 'ESP': 'ES', 'EST': 'EE', 'FIN': 'FI',
    'FRA': 'FR', 'GBR': 'GB', 'GRC': 'GR', 'HRV': 'HR', 'HUN': 'HU',
    'IRL': 'IE', 'ISR': 'IL', 'ITA': 'IT', 'LBY': 'LY', 'LTU': 'LT',
    'LUX': 'LU', 'LVA': 'LV', 'MAR': 'MA', 'MDA': 'MD', 'MKD': 'MK',
    'MLT': 'MT', 'MNE': 'ME', 'NLD': 'NL', 'NOR': 'NO', 'POL': 'PL',
    'PRT': 'PT', 'PSE': 'PS', 'ROU': 'RO', 'RUS': 'RU', 'SAU': 'SA',
    'SRB': 'RS', 'SVK': 'SK', 'SVN': 'SI', 'SWE': 'SE', 'TUN': 'TN',
    'TUR': 'TR', 'UKR': 'UA', 'XKX': 'XK'
}
df['source'] = df['source_country_code'].map(iso3_to_iso2)
df['target'] = df['target_country_code'].map(iso3_to_iso2)
df = df.dropna(subset=['source', 'target'])
df['c1'] = df[['source', 'target']].min(axis=1)
df['c2'] = df[['source', 'target']].max(axis=1)
ntc_df = df.groupby(['c1', 'c2'])['NTC_2030_MW'].mean().round().astype(int).reset_index()

## Process Pypsa-Eur Data 

In [ ]:
lines = n.lines.copy()
lines['country0'] = lines['bus0'].map(n.buses['country'])
lines['country1'] = lines['bus1'].map(n.buses['country'])
cross_lines = lines.query("country0 != country1")
cross_lines['c1'] = cross_lines[['country0', 'country1']].min(axis=1)
cross_lines['c2'] = cross_lines[['country0', 'country1']].max(axis=1)
line_caps = cross_lines.groupby(['c1', 'c2'])['s_nom'].sum()
links = n.links.copy()
links['country0'] = links['bus0'].map(n.buses['country'])
links['country1'] = links['bus1'].map(n.buses['country'])
cross_links = links.query("country0 != country1")
cross_links_dc = cross_links.query("carrier == 'DC'")
cross_links_dc['c1'] = cross_links_dc[['country0', 'country1']].min(axis=1)
cross_links_dc['c2'] = cross_links_dc[['country0', 'country1']].max(axis=1)
link_caps = cross_links_dc.groupby(['c1', 'c2'])['p_nom'].sum()

## Merge

In [ ]:
total_caps = line_caps.add(link_caps, fill_value=0)
total_caps = total_caps.round()
total_caps = total_caps.astype(int)
ntc_pairs = ntc_df[['c1', 'c2']]
total_caps_pairs = total_caps.reset_index()[['c1', 'c2']]

all_pairs = pd.concat([ntc_pairs, total_caps_pairs])
all_pairs = all_pairs.drop_duplicates()
all_pairs = all_pairs.sort_values(['c1', 'c2'])

df_comp = all_pairs.merge(ntc_df, on=['c1', 'c2'], how='left')
df_comp = df_comp.fillna(0)

total_caps_df = total_caps.reset_index()
total_caps_df = total_caps_df.rename(columns={0: 'PyPSA_MW'})
df_comp = df_comp.merge(total_caps_df, on=['c1', 'c2'], how='left')
df_comp = df_comp.fillna(0)
focus_iso2 = ['CZ', 'DE', 'GR', 'IT', 'NL', 'PL']
df_comp = df_comp.query('c1 in @focus_iso2 | c2 in @focus_iso2')
def adjust_row(row):
    if row['PyPSA_MW'] > 1.99 * row['NTC_2030_MW']:
        return row['PyPSA_MW'] / 2
    else:
        return row['PyPSA_MW']
df_comp['PyPSA_MW'] = df_comp.apply(adjust_row, axis=1)
df_comp = df_comp.query('NTC_2030_MW > 0 & PyPSA_MW > 0')
df_comp.to_csv('comparison.csv', index=False)

##

## Plot 

In [ ]:

df_comp['Pair'] = df_comp['c1'] + '-' + df_comp['c2']
fig, ax = plt.subplots(figsize=(20, 8))
x = range(len(df_comp))
width = 0.35
ax.bar([p - width/2 for p in x], df_comp['NTC_2030_MW'], width, label='NTC 2030 Avg')
ax.bar([p + width/2 for p in x], df_comp['PyPSA_MW'], width, label='PyPSA')
ax.set_ylabel('MW')
ax.set_title('Comparison of NTC and PyPSA Capacities')
ax.set_xticks(x)
ax.set_xticklabels(df_comp['Pair'], rotation=90)
ax.legend()
plt.tight_layout()
plt.savefig('comparison_bar.png')
plt.show()